In [20]:
from autogluon.tabular import TabularPredictor
import pandas as pd

# Eleccion de tasas de interes

El objetivo de este notebook es suferir tasas de interes basandonos en el modelo, por lo tanto verifica que tu modelo este bien calibrado, de no estarlo justifica que podrias hacer y que implicaciones tendria. Recuerda que es un escenario real, por lo que debes justificarselo al banco.

Elige unicamente un modelo para realizar esta tarea, y justifica por que elgiste este modelo, puede ser interpretable, no interpretable o lo que tu quieras, sin embargo justifica por que lo elegiste.

Justifica que sets vas a utilizar train/validation/test o algun otro para calcular la calibracion o para proponer las tasas de interes.

# Cargar modelo de Autolguon

Si no sabes como cargar el modelo(s) que entrenaste  puedes revisar la documentacion https://auto.gluon.ai/stable/api/autogluon.tabular.TabularPredictor.load.html 

Para cargar el modelo, debes haberlo salvado (clonado) previamente.

In [29]:
rf = TabularPredictor.load('RandomForest')
test_df = pd.read_parquet('test_df.parquet')


# Calibracion

En esta seccion deberas revisar la calibracion de tu modelo para sustentar que se utilice para generar tasas de interes, que necesitan probabilidad (no class scores).

Coloca graficas o tablas para medir la calibracion de tu modelo. Recuerda explicar que esta pasando, justificar por que esta bien calibrado o no. Y en caso de no estarlo explicar cual es el efecto de que no esta calibrado y que podrias hacer para calibrarlo si te dieran mas tiempo

In [32]:
test_proba = rf.predict_proba(test_df)
test_proba

,0,1
55741,1.000000,7.353054e-09
35406,0.965847,3.415258e-02
66066,0.980403,1.959728e-02
10788,0.863231,1.367685e-01
70834,0.941925,5.807480e-02
...,...,...
17331,0.554168,4.458316e-01
19490,0.687403,3.125972e-01
24995,1.000000,7.353054e-09
46439,0.989250,1.075007e-02


In [34]:
test_df['payment_proba'] = test_proba[0]

# Tasa de Interes

En este apartado deberas de crear recomendaciones, funciones y politicas para sugerir tasas de interes al banco basandote en los modelos. Ten cuidado en que sets utilizas para evaluar tus politicas, al final del dia lo que mas le importa al banco es que tu aproximacion sea real.

Explica detalladamente por que elegiste cada politica en especifico, por que resuelve el problema que planteamos.

`Explicacion`  
La aproximacion mas sencilla del valor esperado del prestamo **como lo vimos en las sesiones de clase** es:  
`E[pago] = Prob(pago)*(1 + interes)Prestamo + P(Impago)(0)` El (0) viene de que no hubo pago. para simplificar asumimos que no paga ningun mes.     
`E[ganancia] =  Prob(pago)*(1 + interes)*Prestamo + P(Impago)(0) - Prestamo` Tenemos que descontar el prestamo inicial para calcular la ganancia.  
  
Si definimos:  
+ Probabilidad de Impago: `p`
+ Probabilidad de Pago := `1-p`
+ Interes := `r`
+ Prestamo := `M`  
+ Valor Esperado de la Ganancia (`E[Ganancia] = E[Pago] - M`)  := `E`
+ Multiplicador de Ganancias:= `Q`, de tal manera que si se quiere ganar Q veces el principal entonces seria `QM`. Para `ganancias igual a cero` tenemos que `E=0M` (`Q=0`), y para `1.5 veces el principal` (osea .5 de ganancia sobre el principal) seria `E=.5M` con `Q=.5`.

Entonces tenemos que:  
`E = (1-p)(1+r)M - p(0) - M`    
`E = (1 + r - p - pr)M - M`    
`E = M(1 + r - p - pr - 1)`  
`E = M(r - p - pr)`    
`E/M = r - pr - p`  
`E/M + p = r(1-p)`  
`1/(1-p)(E/M + p) = r`  
  
Ahora si sustituimos `E` por `QM` tenemos que:  
`1/(1-p)(QM/M + p) = r`  
`1/(1-p)(Q + p) = r`  
`(Q + p)/(1-p) = r`  


Si el valor esperado del prestamo es cero (`E=0=0M` con `Q=0`), entonces:  
`p/(1-p) = r`  
  
Si el valor esperado dede ganancia del prestamo es `Q=.5` (pago total de 1.5 del inicial, menos el dinero prestado `1.5M - M` ) veces el prestamo inicial `QM = .5M`, entonces:  
`(.5 + p)/(1-p) = r`  
 


En estricto sentido esto es solo una aproximacion pues asume que la probabilidad de impago es la cantidad de meses que va a pagar, sin interes acumulado.

 Ademas no olvidar que lo que arroja el modelo no son probabilidades, sino `class scores` pero por simplicidad asumamos que son  probabilidades.  

### **Problemas Reales**  

**Calcular mal este valor esperado podria causar perdidas millonarias al banco, saber utilizar los modelos es tan importe como saber entrenarlos.**


## Escenario 1

Valor esperado de la ganancia del prestamo igual a cero, osea que si se realiza el prestamo el valor esperado de los pagos realizados a la persona sea de 100%. Basicamente que no se peirda ni se gane con el prestamo.


In [35]:
# Calcular la tasa de interés para cada préstamo
test_df['r_escenario_1'] = 1/test_df['payment_proba'] -1
test_df['r_escenario_1'].describe()

count    1.722800e+04
mean     1.888658e+00
std      8.410638e+00
min      7.353054e-09
25%      2.984714e-03
50%      2.241970e-02
75%      3.337372e-01
max      2.083508e+02
Name: r_escenario_1, dtype: float64

In [36]:
# Calculate the expected return for each loan by multiplying the predicted probability of repayment by 1.2
expected_return = test_df['payment_proba'] * (test_df['loan_amnt'] * (1 + test_df['r_escenario_1']))
# Calculate the total expected return as a percentage of the total loan amount
t_r = expected_return.sum() / test_df['loan_amnt'].sum()

# Print the total expected return as a percentage
print("Total expected return: {:.2f}%".format(t_r * 100))

Total expected return: 100.00%


Explicación: Necesitas una tasa de interés de 1/P(pago) - 1 para tener un retorno del 100%

## Escenario 2

Que el valor esperado del prestamo sea igual a 120% o(20% de ganancia). Osea que dadas las probabilidades del modelo el valor esperado del retorno del prestamo sea 20% (o 20% mas que cuando el valor esperado de la ganancia del prestamo era 0%

In [37]:
# Calcular la tasa de interés para cada préstamo
test_df['r_escenario_2'] = 1.2/test_df['payment_proba'] -1
test_df['r_escenario_2'].describe()

count    17228.000000
mean         2.466389
std         10.092765
min          0.200000
25%          0.203582
50%          0.226904
75%          0.600485
max        250.221001
Name: r_escenario_2, dtype: float64

In [38]:
expected_return = test_df['payment_proba'] * (test_df['loan_amnt'] * (1 + test_df['r_escenario_2']))
# Calculate the total expected return as a percentage of the total loan amount
t_r = expected_return.sum() / test_df['loan_amnt'].sum()

# Print the total expected return as a percentage
print("Total expected return: {:.2f}%".format(t_r * 100))

Total expected return: 120.00%


Explicación: Necesitas una tasa de interés de 1.2/P(pago) - 1 para tener un retorno del 100%

## Escenario 3

Que el numero maximo de personas que caigan en impago sea de 10%. Osea que al realizar el prestamo a lo mas el 10% de las personas no pague.

In [40]:
test_df['loan'] = test_df['payment_proba'] > 0.1
test_df['loan']

55741    True
35406    True
66066    True
10788    True
70834    True
         ... 
17331    True
19490    True
24995    True
46439    True
54549    True
Name: loan, Length: 17228, dtype: bool

In [42]:
proporcion_false = test_df['loan'].value_counts(normalize=True).get(False, 0)

print(f"Proporción de registros con valor False en 'loan': {proporcion_false:.2%}")

Proporción de registros con valor False en 'loan': 6.07%


Explicacion: No hay que prestarle a los clientes que tienen probabilidad de pago menor al 10%.

# Explicacion

Explica detalladamente los problemas que podria tener tu modelo y los supuestos que decidiste tomar para simplificar el ejercicio y que implicaciones tiene en la realidad.

Limitaciones:

    Los datos de entrenamiento no son sensibles a cambios en el mundo real como la inflación, crisis económica, etc... Habría que tener un modelo más robusto y con variables que puedan tomar en cuenta el estado económico del sitio en el que va a estar el banco.

    No estoy tomando en cuenta que la tasa de interés que le proponga al cliente va a afectar en su toma de decisión de aceptar el préstamo o irse con otro banco. Esto afecta ya que si detecto que un cliente tiene una probabilidad de impago grande, le ofreceré una tasa de interés muy grande y por lo tanto ya no acepte el préstamo y lo intente con otro banco.